# LangChain Basics: Getting Started

## Introduction

**LangChain** is a framework for developing applications powered by language models. It provides abstractions and tools to build complex LLM applications through composable components.

### What is LangChain?

LangChain enables developers to:
- **Connect LLMs to external data sources** (RAG - Retrieval Augmented Generation)
- **Build agents** that can use tools and make decisions
- **Create complex workflows** by chaining together multiple components
- **Maintain conversation memory** across interactions
- **Switch between providers** (OpenAI, Anthropic, Google) with minimal code changes

### Why LangChain in 2025?

- **Provider-agnostic**: Write once, switch between OpenAI, Anthropic, Google easily
- **Production-ready patterns**: Built-in best practices for RAG, agents, memory
- **Modern architecture**: LangGraph for complex workflows, LCEL for composability
- **Rich ecosystem**: 100+ integrations with vector stores, tools, APIs

### When to Use LangChain

| ✅ Use LangChain For | ❌ Don't Use For |
|---------------------|------------------|
| RAG applications | Simple single LLM calls |
| Multi-step workflows | Maximum performance critical paths |
| Agents with tools | Static predefined responses |
| Provider flexibility | When you only use one provider |
| Complex state management | Simple scripts |

---

## Installation

Install the core LangChain package and provider-specific packages:

In [1]:
# Install LangChain core and providers
# Run in terminal:
# pip install langchain langchain-openai langchain-anthropic langchain-community

# Verify installation
import langchain
print(f"LangChain version: {langchain.__version__}")

LangChain version: 1.0.5


---

## Core Concepts

### 1. Runnables: The Foundation

Everything in LangChain implements the **Runnable** interface with these methods:

```python
.invoke(input)      # Synchronous execution
.stream(input)      # Streaming results
.batch(inputs)      # Batch processing
.ainvoke(input)     # Async execution
```

### 2. Core Components

- **Models**: LLMs and Chat Models (OpenAI, Anthropic, Google, etc.)
- **Prompts**: Templates for structuring inputs
- **Output Parsers**: Extract structured data from LLM responses
- **Chains**: Combine components into workflows (using LCEL)
- **Agents**: Let LLMs decide which tools to use
- **Memory**: Maintain conversation context
- **Retrievers**: Fetch relevant documents (for RAG)

### 3. LCEL (LangChain Expression Language)

Modern way to compose components using the pipe operator `|`:

```python
chain = prompt | model | output_parser
```

Similar to Unix pipes - intuitive and composable!

---

## First Example: "Hello World"

Let's start with a simple LLM call using different providers:

In [2]:
import os
from getpass import getpass

# Set API keys (you'll need at least one)
# Option 1: Set in environment
# export OPENAI_API_KEY="your-key-here"
# export ANTHROPIC_API_KEY="your-key-here"

# Option 2: Set in code (for testing only!)
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API Key: ")

### Example 1: Simple LLM Call with OpenAI

In [3]:
from langchain_openai import ChatOpenAI

# Initialize the model
llm = ChatOpenAI(model="gpt-4", temperature=0.7)

# Simple invoke
response = llm.invoke("Explain LangChain in one sentence")
print(response.content)

LangChain is a blockchain-based language service platform that utilizes artificial intelligence for automated translation.


### Example 2: Same Code, Different Provider

In [4]:
from langchain_anthropic import ChatAnthropic

# Switch to Anthropic Claude - same interface!
llm_claude = ChatAnthropic(model="claude-sonnet-4-5-20250929", temperature=0.7)

# Exact same method call
response = llm_claude.invoke("Explain LangChain in one sentence")
print(response.content)

ModuleNotFoundError: No module named 'langchain_anthropic'

### Key Concept: Provider Independence

Notice how both providers use the same `.invoke()` method? This is the power of LangChain's abstraction - write once, switch providers easily!

---

## Example 3: Streaming Responses

For better UX, stream responses token-by-token:

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4")

# Stream response chunks
for chunk in llm.stream("Write a short poem about Python programming"):
    print(chunk.content, end="", flush=True)

print("\n\n--- Done streaming ---")

In the realm of codes and algorithms divine,
There lies a language, unassuming yet sublime.
Python's the name, simplicity its game,
Turning chaos to harmony, programming's prime.

Less syntax, more action, clear as day,
For both the novice and the expert sway.
Towards problems complex and solutions unique,
With Python at hand, no delay.

Libraries aplenty, versatility utmost,
Data science, machine learning, or web host.
Nestled in its simplicity, a sophisticated beast,
Python, the language, feared by most.

Oh glorious Python, efficient and breezy,
You make the hard tasks seem so easy.
More than just a language, a work of art,
Striking the perfect balance, between light and artsy. 

A programmer's mate, in the digital night,
Guiding through challenges with your inner light.
Python, the unsung hero of our tales,
In lines of code, a world we write.

--- Done streaming ---


---

## Example 4: Simple Chain with LCEL

Let's create our first chain using the pipe operator:

In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 1. Create a prompt template
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")

# 2. Create a model
model = ChatOpenAI(model="gpt-4")

# 3. Create an output parser
output_parser = StrOutputParser()

# 4. Chain them together with the pipe operator!
chain = prompt | model | output_parser

# 5. Use the chain
result = chain.invoke({"topic": "programming"})
print(result)

Why don't programmers like nature?

Because it has too many bugs.


### What Just Happened?

1. **Prompt** receives `{"topic": "programming"}` and creates a formatted message
2. **Model** receives the message and generates a response
3. **Output Parser** extracts the string content from the response

The pipe operator `|` makes it read like a data flow!

---

## Example 5: Chat Messages

Chat models work with messages (system, human, AI):

In [7]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4")

messages = [
    SystemMessage(content="You are a helpful Python programming assistant."),
    HumanMessage(content="How do I read a file in Python?")
]

response = llm.invoke(messages)
print(response.content)

In Python, you can use the built-in `open()` function to read a file. This function takes two parameters: the name of the file, and the mode for which the file should be opened.

The function returns a file object you can use to access the file's contents.

`open()` function gives you several modes for opening files:

- `'r'` (read): for reading a file (default)
- `'w'` (write): for creating a new file and writing to it (will overwrite any existing file with the same name)
- `'a'` (append): for appending to an existing file
- `'b'` (binary): for binary mode, which could be read (`'rb'`), write (`'wb'`), or append (`'ab'`)
- `'t'` (text): for text mode (default, used with `'r'`, `'w'`, and `'a'`)
- `'+'`: for updating (reading and writing)

Here's an example of how you can read a file:

```python
# open the file in read mode
file = open("filename.txt", "r")

# read the file
content = file.read()

# print the content of the file
print(content)

# it's important to close the file when you

### Message Types

- **SystemMessage**: Instructions for the model's behavior
- **HumanMessage**: User input
- **AIMessage**: Model's previous responses (for conversation history)

---

## Example 6: Batch Processing

Process multiple inputs efficiently:

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Create a chain
chain = (
    ChatPromptTemplate.from_template("What is the capital of {country}?")
    | ChatOpenAI(model="gpt-4")
    | StrOutputParser()
)

# Batch process multiple inputs
countries = [{"country": "France"}, {"country": "Japan"}, {"country": "Brazil"}]
results = chain.batch(countries)

for country, capital in zip([c["country"] for c in countries], results):
    print(f"{country}: {capital}")

France: The capital of France is Paris.
Japan: The capital of Japan is Tokyo.
Brazil: The capital of Brazil is Brasília.


---

## Example 7: Async Execution

For non-blocking operations, use async methods:

In [ ]:
import asyncio
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Create chain
chain = (
    ChatPromptTemplate.from_template("Tell me about {topic}")
    | ChatOpenAI(model="gpt-4")
    | StrOutputParser()
)

# Async function
async def get_info(topic):
    result = await chain.ainvoke({"topic": topic})
    return result

# Run multiple requests concurrently
async def main():
    tasks = [
        get_info("Python asyncio"),
        get_info("Python threading"),
        get_info("Python multiprocessing")
    ]
    results = await asyncio.gather(*tasks)
    for i, result in enumerate(results, 1):
        print(f"\nResult {i}:\n{result[:100]}...")

# Run in Jupyter
await main()

---

## Key Takeaways

### ✅ What We Learned

1. **Runnable Interface**: All components share `.invoke()`, `.stream()`, `.batch()`, `.ainvoke()`
2. **Provider Independence**: Switch between OpenAI, Anthropic, Google with minimal code changes
3. **LCEL (Pipe Operator)**: Chain components with `|` for readable workflows
4. **Message Types**: System, Human, AI messages for chat models
5. **Streaming**: Better UX with token-by-token responses
6. **Batch Processing**: Efficient multiple input handling
7. **Async Support**: Non-blocking operations for performance

### 📚 Next Steps

- **langchain_models.ipynb**: Deep dive into different model types and providers
- **langchain_prompts.ipynb**: Advanced prompt engineering techniques
- **langchain_lcel.ipynb**: Master the Expression Language

---

## Practice Exercises

Try these exercises to reinforce your learning:

In [ ]:
# Exercise 1: Create a chain that translates text to a specified language
# Hint: Use ChatPromptTemplate with {text} and {language} variables

# Your code here:
translation_chain = (
    ChatPromptTemplate.from_template("Translate the following text to {language}: {text}")
    | ChatOpenAI(model="gpt-4")
    | StrOutputParser()
)

# Test it
result = translation_chain.invoke({"text": "Hello, how are you?", "language": "Spanish"})
print(result)

In [ ]:
# Exercise 2: Create a streaming chain that generates a story
# Stream the output token by token

# Your code here:
story_chain = (
    ChatPromptTemplate.from_template("Write a short story about {topic}")
    | ChatOpenAI(model="gpt-4")
)

# Stream it
for chunk in story_chain.stream({"topic": "a robot learning to code"}):
    print(chunk.content, end="", flush=True)

In [ ]:
# Exercise 3: Batch process sentiment analysis for multiple texts
# Classify each text as positive, negative, or neutral

# Your code here:
sentiment_chain = (
    ChatPromptTemplate.from_template(
        "Classify the sentiment of this text as positive, negative, or neutral. "
        "Respond with only one word: {text}"
    )
    | ChatOpenAI(model="gpt-4")
    | StrOutputParser()
)

texts = [
    {"text": "I love this product!"},
    {"text": "This is terrible."},
    {"text": "It's okay, nothing special."}
]

results = sentiment_chain.batch(texts)
for text, sentiment in zip(texts, results):
    print(f"{text['text']} -> {sentiment}")

---

## Common Pitfalls

### ❌ Mistake 1: Forgetting to Set API Keys

```python
# This will fail if OPENAI_API_KEY is not set
llm = ChatOpenAI(model="gpt-4")
```

**Solution**: Always check environment variables or pass `api_key` explicitly.

### ❌ Mistake 2: Using Wrong Message Format

```python
# Wrong - ChatModels expect messages, not strings
llm.invoke("Hello")
```

**Solution**: Use proper message types or let prompts handle formatting.

### ❌ Mistake 3: Not Handling Rate Limits

```python
# Batch processing 1000 requests without rate limiting
results = chain.batch([{"input": i} for i in range(1000)])
```

**Solution**: Use `max_concurrency` parameter or implement retry logic.

---

## Resources

- [LangChain Documentation](https://python.langchain.com/)
- [LangChain GitHub](https://github.com/langchain-ai/langchain)
- [LangSmith](https://smith.langchain.com/) - Debugging and monitoring
- [LangChain Blog](https://blog.langchain.dev/)

---

**Next Notebook**: `langchain_models.ipynb` - Deep dive into LLMs, Chat Models, and providers